In [1]:
from scipy.stats import spearmanr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import os
import re

from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%run ../chicm/loader.py
%run ../lib/common_.py

In [21]:
df = pd.read_csv('../data/train.csv')

train, val = train_test_split(df, random_state=23, train_size=.8)

tf = TfidfVectorizer(ngram_range=(1,2))
train_transformed_word = tf.fit_transform(train['answer'] + train['question_body'], )
val_transformed_word = tf.transform(val['answer'] + val['question_body'])

In [22]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1, 4),
    max_features=50000)

In [23]:
train_transformed_char = char_vectorizer.fit_transform(train['answer'] + train['question_body'], )
val_transformed_char = char_vectorizer.transform(val['answer'] + val['question_body'])

In [30]:
model = Ridge()
from scipy.sparse import hstack

model.fit(hstack([train_transformed_char, train_transformed_word]), train['question_not_really_a_question'])

preds = model.predict(hstack([val_transformed_char, val_transformed_word])) 
spearmanr(preds, val['question_not_really_a_question'])

SpearmanrResult(correlation=0.09151392159740285, pvalue=0.0014001362632538909)